In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/News+LPReports/News-Classifiers/summary

/content/gdrive/MyDrive/News+LPReports/News-Classifiers/summary


In [ ]:
! nvidia-smi

Mon Jun 14 01:52:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup

In [1]:
!pip install transformers ipython-autotime sentencepiece sentence-transformers

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import pipeline
import pandas as pd
import torch
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load pre-trained model and data for summarization

In [ ]:
# choose which model you are using
model_name = 't5-base'

# ensure gpu is being utilized
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

model = T5ForConditionalGeneration.from_pretrained(model_name)
model = model.to(device) # send model to gpu

tokenizer = T5Tokenizer.from_pretrained(model_name)

cuda:0


**remember activate gpu**

In [ ]:
df = pd.read_csv('data/100621_grouped.csv')
df.head()

,Search Query,article,signal
0,Aruna,“This facility coming up thanks to the initiat...,revenue
1,Atidot,Mindset Ventures’ previous two funds have resu...,revenue
2,Bicycle Therapeutics,"In the UK and Ireland, the gainers were led by...",revenue
3,Binance,Cryptocurrencies are not here to kill traditio...,revenue
4,Bites,"If you like those warm-weather recipes, you sh...",revenue


In [ ]:
df['words'] = df['article'].apply(lambda x: word_tokenize(x))
df['length'] = df['words'].apply(lambda x: len(x))
df.head(3)

,Search Query,article,signal,words,length
0,Aruna,“This facility coming up thanks to the initiat...,revenue,"[“, This, facility, coming, up, thanks, to, th...",1573
1,Atidot,Mindset Ventures’ previous two funds have resu...,revenue,"[Mindset, Ventures, ’, previous, two, funds, h...",157
2,Bicycle Therapeutics,"In the UK and Ireland, the gainers were led by...",revenue,"[In, the, UK, and, Ireland, ,, the, gainers, w...",122


In [ ]:
# add in code-word for T5 to perform summarization
df['article'] = 'summarize: ' + df['article']
df.head(3)

,Search Query,article,signal,words,length
0,Aruna,summarize: “This facility coming up thanks to ...,revenue,"[“, This, facility, coming, up, thanks, to, th...",1573
1,Atidot,summarize: Mindset Ventures’ previous two fund...,revenue,"[Mindset, Ventures, ’, previous, two, funds, h...",157
2,Bicycle Therapeutics,"summarize: In the UK and Ireland, the gainers ...",revenue,"[In, the, UK, and, Ireland, ,, the, gainers, w...",122


# Run predictions

In [ ]:
# https://huggingface.co/blog/how-to-generate

def summarize(text):
  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True, max_length=512)
  input_ids = input_ids.to(device) # ensure inputs and model on same device

  # generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
  generated_ids = model.generate(input_ids=input_ids, 
                                 min_length=100, 
                                 max_length=300, 
                                 length_penalty=2.0, # set penalty for increasing length (> 1 means force to increase length)
                                 num_beams=4, # reduces the risk of missing hidden high probability word sequences, reduces repeats, makes words more surprising and less probable 
                                #  early_stopping=True,
                                #  no_repeat_ngram_size=4 # no 4-gram appears twice
                                 repetition_penalty = 2.0,
                                #  do_sample=True,
                                #  top_p=0.92,
                                #  top_k=0
                                 )
  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds[0]

In [ ]:
%load_ext autotime
# display time for each cell execution

time: 122 µs (started: 2021-06-10 10:11:18 +00:00)


In [ ]:
df['summary'] = df['article'].apply(lambda x: summarize(x))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


time: 2min 12s (started: 2021-06-10 10:11:18 +00:00)


In [ ]:
print(df['summary'])

0     Aruna Chaudhary launched the women-empowerment...
1     Mindset Ventures’ previous two funds have resu...
2     pharmaceutical company Avadel Pharmaceuticals ...
3     Zhao's claim comes as Binance, the biggest cry...
4     Bagel Bites is giving away acres of the moon d...
                            ...                        
63    a flash loan attack on Binance Smart Chain’s B...
64    Grab will raise its fares by $1 from next mont...
65    companies such as innoviz, hailo and vertilite...
66    Kryon – Raised $53 million to come up with the...
67    OwnBackup generated over $6 million in pipelin...
Name: summary, Length: 68, dtype: object
time: 2.39 ms (started: 2021-06-10 10:13:30 +00:00)


In [ ]:
df['summ_len'] = df['summary'].apply(lambda x: len(word_tokenize(x)))
df.shape

(68, 7)

time: 42.3 ms (started: 2021-06-10 10:13:30 +00:00)


In [ ]:
final_df = df

time: 631 µs (started: 2021-06-10 10:13:30 +00:00)


In [ ]:
# final_df = pd.concat([df, no_summ])
final_df.shape

(68, 7)

time: 3.51 ms (started: 2021-06-10 10:13:30 +00:00)


In [ ]:
final_df.to_csv('output/100621_summary.csv')

time: 25.9 ms (started: 2021-06-10 10:13:30 +00:00)


# NER

In [ ]:
df = pd.read_csv('output/090621_summary_4.csv', index_col=0)
df.head(3)

,Search Query,article,signal,words,length,summary,summ_len
0,Firstcry,summarize: The brand’s official website has se...,revenue,"['The', 'brand', '’', 's', 'official', 'websit...",85,the brand’s official website has seen major fo...,82
1,Grab,"summarize: But for drivers like Mr Joseph Goh,...",revenue,"['But', 'for', 'drivers', 'like', 'Mr', 'Josep...",60,"driver: ""any increase in earnings is good duri...",88
2,Binance,"summarize: Decentralized finance (DeFi), or at...",product,"['Decentralized', 'finance', '(', 'DeFi', ')',...",149,CipherTrace’s transaction tracking and wallet ...,84


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# def ner(text):
#   doc = nlp(text)
#   entity_list = []
#   for ent in doc.ents:
#     if ent.label_ == 'ORG':
#       if ent.text not in entity_list:
#         entity_list.append(ent.text)
  
#   return entity_list

In [ ]:
def ner(text):
  doc = nlp(text)
  entity_list = []
  for ent in doc.ents:
    entity_list.append((ent.text, ent.label_))
  
  return entity_list

In [ ]:
df['entities'] = df['Search Query'].apply(lambda x: ner(x))
df.head()

,Search Query,article,signal,words,length,summary,summ_len,entities
0,Firstcry,summarize: The brand’s official website has se...,revenue,"['The', 'brand', '’', 's', 'official', 'websit...",85,the brand’s official website has seen major fo...,82,[]
1,Grab,"summarize: But for drivers like Mr Joseph Goh,...",revenue,"['But', 'for', 'drivers', 'like', 'Mr', 'Josep...",60,"driver: ""any increase in earnings is good duri...",88,[]
2,Binance,"summarize: Decentralized finance (DeFi), or at...",product,"['Decentralized', 'finance', '(', 'DeFi', ')',...",149,CipherTrace’s transaction tracking and wallet ...,84,[]
3,Flutura,summarize: About Flutura Business Solutions LL...,product,"['About', 'Flutura', 'Business', 'Solutions', ...",179,Flutura is a pioneer in the Industrial IOT Int...,82,[]
4,Grab,summarize: (PHOTO: Getty Images) Superapp Grab...,product,"['(', 'PHOTO', ':', 'Getty', 'Images', ')', 'S...",367,"Grab announced it is partnering with Hao, a su...",91,[]


In [ ]:
df.to_csv('output/100621_summary_3.csv')

# NER HuggingFace

In [ ]:
df = pd.read_csv('output/100621_summary.csv', index_col=0)
df.head(3)

,Search Query,article,signal,words,length,summary,summ_len
0,Aruna,summarize: “This facility coming up thanks to ...,revenue,"['“', 'This', 'facility', 'coming', 'up', 'tha...",1573,Aruna Chaudhary launched the women-empowerment...,78
1,Atidot,summarize: Mindset Ventures’ previous two fund...,revenue,"['Mindset', 'Ventures', '’', 'previous', 'two'...",157,Mindset Ventures’ previous two funds have resu...,79
2,Bicycle Therapeutics,"summarize: In the UK and Ireland, the gainers ...",revenue,"['In', 'the', 'UK', 'and', 'Ireland', ',', 'th...",122,pharmaceutical company Avadel Pharmaceuticals ...,82


time: 27.3 ms (started: 2021-06-10 10:15:08 +00:00)


In [ ]:
nlp = pipeline("ner", device=0) # activate gpu on pipeline
nlp.device

device(type='cuda', index=0)

time: 4.03 s (started: 2021-06-10 10:15:08 +00:00)


In [ ]:
df['ner'] = df['summary'].apply(lambda x: nlp(x))
df.head(3)

,Search Query,article,signal,words,length,summary,summ_len,ner
0,Aruna,summarize: “This facility coming up thanks to ...,revenue,"['“', 'This', 'facility', 'coming', 'up', 'tha...",1573,Aruna Chaudhary launched the women-empowerment...,78,"[{'word': 'A', 'score': 0.9991553425788879, 'e..."
1,Atidot,summarize: Mindset Ventures’ previous two fund...,revenue,"['Mindset', 'Ventures', '’', 'previous', 'two'...",157,Mindset Ventures’ previous two funds have resu...,79,"[{'word': 'Mind', 'score': 0.9991138577461243,..."
2,Bicycle Therapeutics,"summarize: In the UK and Ireland, the gainers ...",revenue,"['In', 'the', 'UK', 'and', 'Ireland', ',', 'th...",122,pharmaceutical company Avadel Pharmaceuticals ...,82,"[{'word': 'Ava', 'score': 0.9995716214179993, ..."


time: 1.99 s (started: 2021-06-10 10:15:12 +00:00)


In [ ]:
def extract_org(a_list):
  string = ''
  # keep the entities with the label 'Organization'
  for i in a_list:
    if i['entity'] == 'I-ORG':
      entity = ' ' + i['word']
      string += entity
  
  # join all those words with ##, it is Huggingface's way of saying it came from the same entity
  string = string.replace(' ##', '')
  return string

time: 3.97 ms (started: 2021-06-10 10:15:14 +00:00)


In [ ]:
df['entity'] = df['ner'].apply(lambda x: extract_org(x))
df.head(3)

,Search Query,article,signal,words,length,summary,summ_len,ner,entity
0,Aruna,summarize: “This facility coming up thanks to ...,revenue,"['“', 'This', 'facility', 'coming', 'up', 'tha...",1573,Aruna Chaudhary launched the women-empowerment...,78,"[{'word': 'A', 'score': 0.9991553425788879, 'e...",
1,Atidot,summarize: Mindset Ventures’ previous two fund...,revenue,"['Mindset', 'Ventures', '’', 'previous', 'two'...",157,Mindset Ventures’ previous two funds have resu...,79,"[{'word': 'Mind', 'score': 0.9991138577461243,...",Mindset Ventures Prodigy Upstart DiA Philips ...
2,Bicycle Therapeutics,"summarize: In the UK and Ireland, the gainers ...",revenue,"['In', 'the', 'UK', 'and', 'Ireland', ',', 'th...",122,pharmaceutical company Avadel Pharmaceuticals ...,82,"[{'word': 'Ava', 'score': 0.9995716214179993, ...",Avadel Pharmaceuticals AVDL Rio Tinto Group R...


time: 52 ms (started: 2021-06-10 10:15:14 +00:00)


In [ ]:
idx_list = []
for idx,row in df.iterrows():
  if row['Search Query'].lower() in row['entity'].lower():
    idx_list.append(idx)
print(idx_list)

[1, 2, 3, 4, 7, 8, 9, 11, 15, 16, 17, 22, 24, 25, 27, 28, 30, 31, 32, 33, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 47, 49, 50, 52, 54, 55, 57, 58, 59, 62, 63, 64, 66, 67]
time: 11.2 ms (started: 2021-06-10 10:15:14 +00:00)


In [ ]:
final_df = df.iloc[idx_list]
final_df.head(3)

,Search Query,article,signal,words,length,summary,summ_len,ner,entity
1,Atidot,summarize: Mindset Ventures’ previous two fund...,revenue,"['Mindset', 'Ventures', '’', 'previous', 'two'...",157,Mindset Ventures’ previous two funds have resu...,79,"[{'word': 'Mind', 'score': 0.9991138577461243,...",Mindset Ventures Prodigy Upstart DiA Philips ...
2,Bicycle Therapeutics,"summarize: In the UK and Ireland, the gainers ...",revenue,"['In', 'the', 'UK', 'and', 'Ireland', ',', 'th...",122,pharmaceutical company Avadel Pharmaceuticals ...,82,"[{'word': 'Ava', 'score': 0.9995716214179993, ...",Avadel Pharmaceuticals AVDL Rio Tinto Group R...
3,Binance,summarize: Cryptocurrencies are not here to ki...,revenue,"['Cryptocurrencies', 'are', 'not', 'here', 'to...",1837,"Zhao's claim comes as Binance, the biggest cry...",85,"[{'word': 'Zhao', 'score': 0.9957382678985596,...",Binance BSC Binance BSC


time: 51.3 ms (started: 2021-06-10 10:15:14 +00:00)


In [ ]:
final_df.drop(columns=['words','length','summ_len','ner','entity'], inplace=True)
final_df.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Search Query,article,signal,summary
1,Atidot,summarize: Mindset Ventures’ previous two fund...,revenue,Mindset Ventures’ previous two funds have resu...
2,Bicycle Therapeutics,"summarize: In the UK and Ireland, the gainers ...",revenue,pharmaceutical company Avadel Pharmaceuticals ...
3,Binance,summarize: Cryptocurrencies are not here to ki...,revenue,"Zhao's claim comes as Binance, the biggest cry..."


time: 17.3 ms (started: 2021-06-10 10:15:14 +00:00)


In [ ]:
final_df.to_csv('output/100621_summary_hf.csv')

time: 12.9 ms (started: 2021-06-10 10:15:14 +00:00)


# Semantic similarity

In [ ]:
df = pd.read_csv('output/110621_after_summary.csv', index_col=0)
df.head(3)

,Search Query,article,URL,signal,summary
0,17Live,summarize: Part of the relief act could be use...,['https://www.taipeitimes.com/News/editorials/...,revenue,teacher used livestreaming app 17Live to teach...
1,Binance,summarize: It is already impossible for a sing...,['https://techstory.in/bitcoin-cannot-be-shut-...,revenue,"Changpeng ""CZ"" Zhao is the chief executive of ..."
2,Elevation Oncology,summarize: It has a strategic alliance agreeme...,['https://www.directorstalkinterviews.com/neog...,revenue,the company was founded in 2001 and is headqua...


In [ ]:
model = SentenceTransformer('stsb-roberta-large')

In [ ]:
# clean quotation marks from "" to '', or else model will throw error
df['summary'] = df['summary'].apply(lambda x: x.replace('"', "'"))
df.head(3)

,Search Query,article,URL,signal,summary
0,17Live,summarize: Part of the relief act could be use...,['https://www.taipeitimes.com/News/editorials/...,revenue,teacher used livestreaming app 17Live to teach...
1,Binance,summarize: It is already impossible for a sing...,['https://techstory.in/bitcoin-cannot-be-shut-...,revenue,Changpeng 'CZ' Zhao is the chief executive of ...
2,Elevation Oncology,summarize: It has a strategic alliance agreeme...,['https://www.directorstalkinterviews.com/neog...,revenue,the company was founded in 2001 and is headqua...


In [ ]:
# this function returns the semantic similarity between 2 texts
def similar_sentence(sentence1, sentence2):
  embedding1 = model.encode(sentence1, convert_to_tensor=True)
  embedding2 = model.encode(sentence2, convert_to_tensor=True)
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  return cosine_scores

In [ ]:
# get the list of unique companies to iterate through them
unique_list = df['Search Query'].unique()
print(unique_list)

['17Live' 'Binance' 'Elevation Oncology' 'Firstcry' 'Grab'
 'Innoviz Technologies' 'Kissht' 'Kryon' 'Licious' 'Nium' 'OwnBackup'
 'Payfazz' 'ScyllaDB' 'Signzy' 'Synup' 'TaniHub' 'Valtix' 'Verbit'
 'Xpressbees' 'Atidot' 'Flutura' 'GlowRoad' 'Bicycle Therapeutics'
 'Hotelogix' 'CODA' 'Taranis']


In [ ]:
# this function utilizes semantic similarity to remove "similar" summaries across the dataframe
def remove_dups(unique_list, df):
  idx_list = []
  for name in unique_list:
    # get each group of dataframe by unique company name
    small_df = df[df['Search Query'] == name].reset_index()

    for idx,row in small_df.iterrows():
      text1 = row['summary']

      # compare each row with every other next row in the dataframe
      while idx < len(small_df) - 1:
        idx += 1
        # keep old index for reference to original df
        index2 = small_df.loc[idx, 'index']
        text2 = small_df.loc[idx, 'summary']
        score = similar_sentence(text1, text2)

        # if the semantics are more than 70% similar, append the old index to a list for dropping
        if score > 0.7:
          idx_list.append(index2)

  return idx_list

In [ ]:
idx_list = remove_dups(unique_list, df)

In [ ]:
df2 = df.drop(df.index[idx_list])
df2.head()

,Search Query,article,URL,signal,summary
0,17Live,summarize: Part of the relief act could be use...,['https://www.taipeitimes.com/News/editorials/...,revenue,teacher used livestreaming app 17Live to teach...
1,Binance,summarize: It is already impossible for a sing...,['https://techstory.in/bitcoin-cannot-be-shut-...,revenue,Changpeng 'CZ' Zhao is the chief executive of ...
2,Elevation Oncology,summarize: It has a strategic alliance agreeme...,['https://www.directorstalkinterviews.com/neog...,revenue,the company was founded in 2001 and is headqua...
3,Firstcry,"summarize: Earlier, voting for Indian Idol 12 ...",['https://inc42.com/buzz/ananth-narayanans-thr...,revenue,voting for Indian Idol 12 used to happen on th...
4,Grab,summarize: Grab announced the launch of GrabSu...,['https://vulcanpost.com/747587/grabsupermarke...,revenue,Grab announced the launch of GrabSupermarket i...


In [ ]:
df2.shape

(40, 5)

In [ ]:
df2 = df2.sort_values(by=['Search Query'])
df2.to_csv('output/140621_no_dup.csv')